In [ ]:
# default_exp functional

In [ ]:
#export
from kesscore.imports import *
from decorator import decorator
from functools import wraps

In [ ]:
#export
def _wrapify(f, t):
    @wraps(f)
    def _inner(*args,**kwargs): return t(f(*args,**kwargs))
    return _inner
_listify = partial(_wrapify, t=list)
_Listify = partial(_wrapify, t=L)

In [ ]:
#export
lmap = _listify(map)
lzip = _listify(zip)
lfilter = _listify(filter)
lrange = _listify(range)
Lmap = _Listify(map)
Lzip = _Listify(zip)
Lfilter = _Listify(filter)
Lrange = _Listify(range)

In [ ]:
test_eq_type(lmap(noop, [1,2,3]),          [1,2,3])
test_eq_type(lzip([1,2],[3,4]),            [(1,3),(2,4)])
test_eq_type(lfilter(ge(2), [1,2,3,4]),    [2,3,4])
test_eq_type(lrange(3),                    [0,1,2])
test_eq_type(Lmap(noop, [1,2,3]),          L([1,2,3]))
test_eq_type(Lzip([1,2],[3,4]),            L([(1,3),(2,4)]))
test_eq_type(Lfilter(ge(2), [1,2,3,4]),    L([2,3,4]))
test_eq_type(Lrange(3),                    L([0,1,2]))

In [ ]:
#export
def _all(self:L)->bool: return all(self)
L.all = _all

In [ ]:
assert not L([True, False, True] ).all()
assert not L([True, True,  False]).all()
assert     L([True, True,  True] ).all()

In [ ]:
#export
_isinstance = isinstance
def isinstance(obj, class_or_tuple=None):
    'Like isinstance but support creating partial versions, e.g, isinstance(int)(1) is True.'
    if class_or_tuple is None:
        class_or_tuple = obj
        def _inner(obj): 
            return _isinstance(obj, class_or_tuple)
        _inner.__doc__ = f'Check if `obj` is of type\\s: {class_or_tuple}'
        return _inner
    return _isinstance(obj,class_or_tuple)

In [ ]:
assert isinstance(obj=1, class_or_tuple=int)
assert isinstance(int)(1)
test_fail(lambda: isinstance(1)(int), contains='isinstance() arg 2 must be a type or tuple of types')

In [ ]:
assert isinstance(int)(1)
assert all(map(isinstance((list, int)), [1, 2, 3, [1, 2, 3]]))
assert not isinstance((list, int))((1, 2, 3))

In [ ]:
from nbdev.sync import notebook2script

In [ ]:
notebook2script()

Converted 00_functional.ipynb.
Converted 01_images.ipynb.
Converted 02_fastdl.ipynb.
Converted 03_datadownload.ipynb.
Converted 04_random.ipynb.
Converted 05_tensor.ipynb.
Converted index.ipynb.
